# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [696]:
import pandas as pd
data = pd.read_csv(r'/datasets/data.csv')
data.info()
# Пришлось вставить 'r' перед путем к файлу, потому что иначе он читал это не как путь к файлу, а как строку. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

При первом взгляде на данные мы видим, что у нас есть 21525 строк и 12 столбцов. В колонках days_employed и total_income количество non-null объектов меньше количества строк, следовательно там имеются пропуски. Также в данных имеются разные типы данных: float, int и object. 

## Шаг 2. Предобработка данных

### Обработка пропусков

In [697]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Количество пропусков в столбцах стажа и дохода равны и соответствуют строкам. В остальных столбцах пропусков нет

In [698]:
median_income_grouped = data.groupby('income_type')['total_income'].median()
median_income = pd.DataFrame({'median_income':median_income_grouped}).reset_index()


Создал датафрейм с медианными значениями доходов по каждой группе типа занятости. С помощью reset_index() убрал income type из индекса. Медиана нужна, если среднее значение некорректно характеризует данные, когда некоторые значения 
сильно выделяются среди большинства.

In [699]:
data = data.merge(median_income, on = 'income_type', how = 'left') 



Слияние датафреймов data и median_income по столбцу income_type с индексами data

In [700]:
data['total_income'] = data['total_income'].fillna(data['median_income'])
data['total_income'].isna().sum()

0

Заполнил пустые значения столбца дохода средними значениями из столбца mean_income по типу занятости

# Приступаю к поиску аномальных значений

In [701]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,142594.396847
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,142594.396847
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,142594.396847
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,142594.396847
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,118514.486412
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,172357.950966
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,172357.950966
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,142594.396847
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,142594.396847
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,142594.396847


Здесь мы видим, что в столбце days_employed наблюдаются аномальные значения: отрицательные числа, а также неправдаподобное количество отработанных дней

In [702]:
data['days_employed'] = abs(data['days_employed'])
data['days_employed'].head(20)

0       8437.673028
1       4024.803754
2       5623.422610
3       4124.747207
4     340266.072047
5        926.185831
6       2879.202052
7        152.779569
8       6929.865299
9       2188.756445
10      4171.483647
11       792.701887
12              NaN
13      1846.641941
14      1844.956182
15       972.364419
16      1719.934226
17      2369.999720
18    400281.136913
19     10038.818549
Name: days_employed, dtype: float64

Здесь я привел все значения столбца days_employed к модулю, так как в данных встречались и отрицательные значения со знаком минус.

In [703]:
data.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

Сгруппировав по типу занятости, высчитал медиану трудового стажа для каждой группы. Данные, превышающие 360 тысяч дней, кажутся неправдоподобными, даже если они копили на квартиру в центре Москвы и работали по 156 часов в сутки. Предполагаю, что в этих группах трудовой стаж указан в часах, а не в днях.

In [704]:
median_days_grouped = data.groupby('income_type')['days_employed'].median() 
median_days = pd.DataFrame({'median_days':median_days_grouped})
data = data.merge(median_days, on = 'income_type', how = 'left')


In [705]:
data['days_employed'] = data['days_employed'].fillna(data['median_days'])
data['days_employed'].isna().sum()


0

In [706]:
try:
    pd.to_numeric(data['days_employed'])
except: 
    display('empty')
data['days_employed'].describe

<bound method NDFrame.describe of 0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64>

In [707]:
data = data.loc[data['income_type'] != 'безработный']
data.groupby('income_type')['days_employed'].mean()

income_type
в декрете            3296.759962
госслужащий          3328.308350
компаньон            2055.165652
пенсионер          365025.963652
предприниматель       520.848083
сотрудник            2251.736421
студент               578.751554
Name: days_employed, dtype: float64

Трудовой стаж у безработных тоже привышает 350 тысяч дней. А в данных всего две строки, относящиеся к категории "безработный". По двум строкам анализировать и делать какие-то выводы по категории - бессмысленно, поэтому мы их удалим.

In [708]:
data['retire_emp'] = data.loc[data['income_type'] == 'пенсионер']['days_employed']/24
data.loc[data['income_type'] == 'пенсионер', ['days_employed']] = data.loc[data['income_type'] == 'пенсионер']['retire_emp']
data = data.drop(['retire_emp', 'median_days', 'median_income'], axis =1)
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [709]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Я заполнил пропуски в столбцах дохода и трудового стажа медианными значениями, которые вывел в ходе группировки по типу занятости. Аномальные значения по этим двум колонкам, где имелись пропуски, были обнаружены в колонке трудового стажа: отрицательное количество дней (знак "-" перед цифрами), а также количество дней трудового стажа в категории "пенсионер", превышающее 360 тысяч дней. По моему мнению, трудовой стаж был записан в часах, поэтому я разделил значения на 24, чтобы привести эти значения к дням. Средний трудовой стаж для пенсионеров составил порядка 15 тысяч дней, что равняется около 40-45 годам трудового стажа. Таким образом, эти данные стали выглядеть более логично и правдоподобно. Помимо этого, я очистил данные от категории "безработные", потому что там были всего 2 строки по этому типу занятости. А делать объективный анализ всего лишь по двум строкам неправильно.

### Замена типа данных

In [710]:
for_toint = ['children', 'days_employed', 'dob_years', 'total_income']
for i in for_toint:
    data[i] = data[i].astype('int')

**Вывод**

С помощью цикла for заменил вещественные типы данных на целочисленные в тех столбцах, которые являются количественными, а не категориальными. Использовал метод .astype, т.к. pd.to_numeric меняет тип данных на "float". В случае, если бы я не заполнил пропуски трудового стажа медианным значением, то можно было бы сначала использовать to_numeric с errors = 'coerce', а далее методом try-except опять же с помощью .astype() перевести в целочисленные значения.

### Обработка дубликатов

In [711]:
data['education'].value_counts()

среднее                13749
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   267
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

После анализа количества уникальных значений в категориальных столбцах, я обнаружил, что только в столбце 'education' есть дубликаты, возникшие из-за различий в регистре. Возможно, данные об образовании заполняются вручную сотрудниками, либо клиентами, поэтому и возникли различия в регистре.

In [712]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15232
высшее                  5259
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

**Вывод**

Мы избавились от неявных дублей. Но остались явные

In [713]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

Оказался невнимательным, и забыл избавиться от явных дубликатов с помощью метода drop_duplicates(). Reset_index использовал для того, чтобы в индексах не было пропусков, drop=True, чтобы не создавать столбец со старыми значениями индексов.

### Лемматизация

In [714]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem() 
purp_unique = data['purpose'].unique()
lemmas_list = []
for i in purp_unique:
    lemmas_list.append(m.lemmatize(i))
lemmas = []
for j in range(len(lemmas_list)):
    for k in lemmas_list[j]:
        lemmas.append(k)
display(Counter(lemmas))

Counter({'покупка': 10,
         ' ': 59,
         'жилье': 7,
         '\n': 38,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 1,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'подержанный': 1,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1})

**Вывод**

Количество лемм в уникальных значениях столбца purpose: "жилье" - 7, "недвижимость" - 10, "автомобиль" - 9, "свадьба" - 3, "образование" - 9. 
    <br /> Я импортировал Mystem присвоив переменной m его функции. Также импортировал Counter для подсчета. В переменную записал все уникальные значения столбца purpose, и сначала с помощью цикла добавил все леммы в список lemmas_list. Далее для того, чтобы привести все это в корректный для лемматизации формат, я использовал вложенный цикл для двумерного списка lemass_list, лемматизировал новый список lemmas и подсчитал значения лемм.

### Категоризация данных

In [715]:
family_df = data[['family_status_id', 'family_status']]
education_df = data[['education_id', 'education']]
data = data.drop(columns = ['family_status', 'education'])



Созданы два новых датафреймами со столбцами id и значений по семейному статусу и уровню образования. Из основного датафрейма удалены столбцы family_status и education и оставлены только колонки с их идентификаторами. 
   

In [716]:
low_inc = data.query('total_income < 120000')
med_inc = data.query(('total_income > 120000')and('total_income < 180000'))
high_inc = data.query('total_income > 180000')



Провел категоризацию по уровню дохода, где до 120 тысяч - это низкий уровень дохода, между 120 и 180 тысяч - средний, а больше 180 тысяч - это выский уровень дохода

In [717]:
data['children'] = abs(data['children'])
data.loc[data['children'] == 20, 'children'] = 2 
childfree = data.query('children == 0')
child_have = data.query('children > 0')

По колонке children разделил на 2 колонки: есть дети и нет детей. Так как количество заемщиков с количеством детей > 3 очень мало, анализ получился бы не совсем корректным. Также 20 детей, по моему мнению, какая-то ошибка, возможно, вследствие неправильной записи. Возможно, подразумевалось 2 ребенка, поэтому всю категорию добавил к группе с 2-мя детьми.

In [718]:
married = data.query('family_status_id == 0')
civil = data.query('family_status_id == 1')
widowed = data.query('family_status_id == 2')
divorced = data.query('family_status_id == 3')
single = data.query('family_status_id == 4')


По семейному положению разделил на категории "в браке", "в гражданском браке", "вдовцы/вдовы", "разведенные", "не в браке". Для фильтрации использовал идентификатор из колонки family_status_id 

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [719]:
childfree_per = ((childfree['debt'].sum())/childfree['debt'].count()) 
child_have_per = ((child_have['debt'].sum())/child_have['debt'].count())
childfree_per = f'{childfree_per:.2%}' 
child_have_per = f'{child_have_per:.2%}'
display( childfree_per, child_have_per)


'7.54%'

'9.20%'

**Вывод**


Я разделил категорию children на 2 группы: "бездетные" и "с детьми", и поделил сумму значений ('debt = 1' - должник. Сумма таких значений - количество должников) на общее количество значений в столбцах debt для каждой категории, тем самым выявил долю недобросовестных заемщиков для категорий "бездетные" и "с детьми". Согласно результатам, доля недобросовестных бездетных заемщиков составляет 7.51%, и меньше чем доля должников в категории "с детьми" более чем на 1.5%. 
    <br /> <b> Вывод: </b>
    <br /> Зависимость между наличием детей и возвратом кредита в срок наблюдается. Общее количество бездетных заемщиков почти в 2 раза больше, чем количество должников в категории "с детьми", однако несмотря на это, недобросовестных заемщиков в категории "с детьми" больше. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [720]:
married_per = ((married['debt'].sum())/married['debt'].count())
divorced_per = ((divorced['debt'].sum())/divorced['debt'].count()) 
civil_per = ((civil['debt'].sum())/civil['debt'].count()) 
widowed_per = ((widowed['debt'].sum())/widowed['debt'].count()) 
single_per = ((single['debt'].sum())/single['debt'].count()) 
married_per = f'{married_per:.2%}' 
divorced_per = f'{divorced_per:.2%}' 
civil_per = f'{civil_per:.2%}' 
widowed_per = f'{widowed_per:.2%}' 
single_per = f'{single_per:.2%}' 
display(married_per, divorced_per, civil_per, widowed_per, single_per)


'7.54%'

'7.11%'

'9.35%'

'6.57%'

'9.75%'

**Вывод**

Здесь я посчитал долю должников для каждой категории семейного статуса: 
    <br /> Вдовец/вдова - 959 человек от общего количества заемщиков. Доля недобросовестных - 6.57%
    <br /> Гражданский брак - 4150 человек от общего количества заемщиков. Доля недобросовестных - 9.35% 
    <br /> В разводе - 1195 человек от общего количества заемщиков. Доля недобросовестных - 7.11%
    <br /> Женаты/замужем - 12338 человек от общего количества заемщиков. Доля недобросовестных - 7.54%
    <br /> Не женаты/не замужем - 2810 человек от общего количества заемщиков. Доля недобросовестных -9.75%
    <br /> <b> Вывод: </b>
    <br /> Наименьшее количество займов, и самая маленькая доля недобросовестных заемщиков у вдовцов/вдов. Самая большая доля недобросовестных заемщиков у категории "не женаты/не замужем" - 9.74% при количестве займов почти в 6 раз меньше, чем у женатых. Здесь явно прослеживается зависимость между семейным положением и возвратом кредита в срок. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [721]:
high_inc_per = ((high_inc['debt'].sum())/high_inc['debt'].count())
med_inc_per = ((med_inc['debt'].sum())/med_inc['debt'].count())
low_inc_per = ((low_inc['debt'].sum())/low_inc['debt'].count())
high_inc_per = f'{high_inc_per:.2%}'
med_inc_per = f'{med_inc_per:.2%}'
low_inc_per = f'{low_inc_per:.2%}'
display(high_inc_per, med_inc_per, low_inc_per)


'7.44%'

'8.40%'

'8.09%'

<br /> Количество заемщиков с высоким уровнем дохода - 6422. 
    <br /> Количество заемщиков со средним уровнем дохода - 15030. 
    <br /> Количество заемщиков с низким уровнем дохода - 7230. 
    <br /> Доля недобросовестных заемщиков с высоким уровнем дохода - 7.44%.
    <br /> Доля недобросовестных заемщиков со средним уровнем дохода - 8.40%
    <br /> Доля недобросовестных заемщиков с уровнем дохода ниже среднего - 8.09%.
    <br /> <b> Вывод: </b>
    <br /> Здесь мы можем наблюдать, что взаимосвязь между уровнем дохода и возвратом кредита в срок наблюдается: люди с высоким уровнем дохода берут кредиты почти в 2.5 раза реже, чем заемщики со средним уровнем дохода, и при этом доля должников в категории с высоким уровнем дохода, почти на 1% меньше, чем у среднего дохода. Разница между замещиками ниже среднего и среднего дохода составляет менее половины процента, при количестве заемщиков различающихся почти в 2 раза. 

- Как разные цели кредита влияют на его возврат в срок?

In [722]:
def purpose_categ (purpose):
    lemma = ' '.join(m.lemmatize(purpose))
 
    if 'авто' in lemma:
        return 'автомобиль'
    if 'недвижимость' in lemma:
        return  'недвижимость'
    if 'жилье' in lemma:
        return  'недвижимость'
    if 'свадьба' in lemma:
        return  'свадьба'
    if 'образование' in lemma:
        return 'образование'
    else:
        return 
 
data['pur_lem'] = data['purpose'].apply(purpose_categ)
auto = data.loc[data['pur_lem'] == 'автомобиль']
prop = data.loc[data['pur_lem'] == 'недвижимость']
wedd = data.loc[data['pur_lem'] == 'свадьба']
educ = data.loc[data['pur_lem'] == 'образование']
auto_per = ((auto['debt'].sum())/auto['debt'].count())
prop_per = ((prop['debt'].sum())/prop['debt'].count())
wedd_per = ((wedd['debt'].sum())/wedd['debt'].count())
educ_per = ((educ['debt'].sum())/educ['debt'].count())
auto_per = f'{auto_per:.2%}'
prop_per = f'{prop_per:.2%}'
wedd_per = f'{wedd_per:.2%}'
educ_per = f'{educ_per:.2%}'
display(auto_per, prop_per, wedd_per, educ_per)


'9.36%'

'7.23%'

'8.00%'

'9.22%'

**Вывод**

<br /> Я разбил займы по категориям "автомобиль", "недвижимость", "образование" и "свадьба":
    <br /> Доля недобросовестных заемщиков в категории "автомобиль" - 9.36%. Общее количество займов по категории - 4315
    <br /> Доля недобросовестных заемщиков в категории "недвижимость" - 7.23%. Общее количество займов по категории - 10840
    <br /> Доля недобросовестных заемщиков в категории "свадьба" - 8.00%. Общее количество займов по категории - 2348
    <br /> Доля недобросовестных заемщиков в категории "образование" - 9.22%. Общее количество займов по категории - 4022
    <br /> <b> Вывод: </b>
    <br /> Зависимость возврата кредита в срок и цели, с которой был осуществлен займ прослеживается. Самыми порядочными заемщиками стали "ипотечники": при наибольшем общем количестве займов (10840), доля недобросовестных заемщиков составляет 7.23%. Любители кататься, как показывают данные, возить саночки не особо любят. Также как и те, кто берет кредиты на обучение. При почти равном количестве займов в категориях "автомобиль" и "образование", доля недобросовестных заемщиков также остается больше, чем в категориях "недвижимость" и "свадьба". 

## Шаг 4. Общий вывод

In [723]:
pur_per_grouped = [auto_per, prop_per, educ_per, wedd_per]
pur = data.groupby('pur_lem').count()
pur['per'] = pur_per_grouped
pur.drop(columns = ['family_status_id', 'children', 'days_employed', 'dob_years', 'education_id', 'gender', 'income_type', 
                       'purpose', 'total_income'])

,debt,per
pur_lem,,
автомобиль,4306,9.36%
недвижимость,10809,7.23%
образование,4013,9.22%
свадьба,2324,8.00%


In [724]:
family_per_grouped = [married_per, civil_per, widowed_per, divorced_per, single_per]
family = data.groupby('family_status_id').count()
family['per'] = family_per_grouped
family.drop(columns = ['children', 'days_employed', 'dob_years', 'education_id', 'gender', 'income_type', 
                       'purpose', 'total_income', 'pur_lem'])


,debt,per
family_status_id,,
0,12338,7.54%
1,4150,9.35%
2,959,6.57%
3,1195,7.11%
4,2810,9.75%


In [725]:
child_data = [{'category' : 'Без детей', 'count': 14149, 'per': '7.54%'}, 
              {'category' : 'С детьми', 'count': 7376, 'per': '9.20%'}]
child_table = pd.DataFrame(child_data)
child_table

,category,count,per
0,Без детей,14149,7.54%
1,С детьми,7376,9.20%


In [726]:
income_data = [{'category' : 'Высокий', 'count': 6422, 'per': '7.44%'},
              {'category' : 'Средний', 'count': 15030, 'per': '8.40%'},
              {'category' : 'Низкий', 'count': 7230, 'per': '8.09%'}]
income_table = pd.DataFrame(income_data)
income_table

,category,count,per
0,Высокий,6422,7.44%
1,Средний,15030,8.40%
2,Низкий,7230,8.09%


<br /><head1><b> Общий вывод: <head1></b>
    <br /> С самого начала мы обнаружили 2174 пропуска в столбцах days_employed и total_income в соответствующих строках, и решили заполнить их медианными значениями. Далее обнаружились аномальные значения в столбце дней трудового стажа: отрицательные значения, и количество дней трудового стажа у пенсионеров, превышающий 350 тысяч дней. По моим предположениям, для пенсионеров были записаны часы трудового стажа, а не дни, соответственно, мы разделили значения в категории "пенсионеры" на 24. Впоследствии, медианный трудовой стаж у пенсионеров составил порядка 15 тысяч дней (40-45 лет). 
    <br />  Далее во всех количественных столбцах мы привели значения от вещественных к целочисленным для дальнейшего анализа. Также избавились от неявных дубликатов в столбце education, и избавились от явных дубликатов. После этого лемматизировали данные по целям кредита и выявили несколько основных категорий: "образование", "недвижимость", "автомобиль", "свадьба". Также провели категоризацию по семейному статусу, уровню дохода и наличию детей.
    <br /> После проведенных над данными операций, я решил рассчитать долю должников по категориям. Согласно данным рассчетам, по моим предположениям, в группу риска попали следующие категории: 
    <br /> По целям: "образование", "автомобиль".
    <br /> По семейному статусу: "не в браке" и "гражданский брак"
    <br /> По уровню дохода: здесь сложно давать однозначную рекомендацию, ведь большая часть заемщиков из категории "средний доход", однако доля должников в этой категории самая высокая. Поэтому при отсутствие факторов риска по другим категориям, можно реализовать оформление займа.
    <br /> По наличию детей: здесь также все неоднозначно. При количестве займов в категории "без детей", почти в 2 раза выше, чем "с детьми", доля должников у первых почти на 2% меньше. Также как и с доходом, здесь при оформлении займа должны учитываться факторы и из других категорий.
